In [2]:
import csv
import random
from collections import defaultdict

In [3]:
LEMMAS = "./polimorfologik-2.1/polimorfologik-2.1.txt"
QUOTES_FILE = "cytaty.txt"
forbiddenWords = ["go", "on", "ich", "ona", "jego", "ono", "jej", "mu", "jemu"]
defaultAnswers = ["Ładna dziś pogoda, prawda?", "Może zmieńmy temat"]

In [8]:
lemmasDict = {}

with open(LEMMAS) as f:
    for line in csv.reader(f, delimiter=';'):
        lemmasDict[line[1]] = line[0]


['AA', 'AA', 'subst:pl:acc:m3+subst:pl:acc:n2+subst:pl:acc:p1+subst:pl:dat:m3+subst:pl:dat:n2+subst:pl:dat:p1+subst:pl:gen:m3+subst:pl:gen:n2+subst:pl:gen:p1+subst:pl:inst:m3+subst:pl:inst:n2+subst:pl:inst:p1+subst:pl:loc:m3+subst:pl:loc:n2+subst:pl:loc:p1+subst:pl:nom:m3+subst:pl:nom:n2+subst:pl:nom:p1+subst:pl:voc:m3+subst:pl:voc:n2+subst:pl:voc:p1+subst:sg:acc:m3+subst:sg:acc:n2+subst:sg:dat:m3+subst:sg:dat:n2+subst:sg:gen:m3+subst:sg:gen:n2+subst:sg:inst:m3+subst:sg:inst:n2+subst:sg:loc:m3+subst:sg:loc:n2+subst:sg:nom:m3+subst:sg:nom:n2+subst:sg:voc:m3+subst:sg:voc:n2']
AA
AA


In [5]:
print(lemmasDict.get())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
quotes = []
index = defaultdict(set)

with open(QUOTES_FILE) as f:
    quoteNum = 0
    for line in f:
        curQuotes = line[2:].lower().replace("&lt;br /&gt", '').replace('!', '.').replace('?', '.').split('.')
        curQuotes = [q for q in curQuotes if len(q) > 3]
        for q in curQuotes:
            words = q.split()
            if len(words) == 0:
                continue
            quotes.append(q)
            for w in words:
                if lemmasDict.get(w) != None:
                    index[lemmasDict[w]].add(quoteNum)
            quoteNum += 1
        #print(curQuotes)



In [5]:
print(len(quotes))
print(len(index))

471898
45894


In [6]:
def searchQuotes(query):
    query = query.lower()
    allPossibilities = set.union(set(), *(index[lemmasDict.get(w)] for w in query.split() if w not in forbiddenWords))
    return [quotes[idx] for idx in allPossibilities]

In [7]:
len(quotes)

471898

In [8]:
searchQuotes("goryle")

["'''generał bogusław góra:''' lepszy z ciebie materiał na trupa niż na goryla",
 'święty bazyli wypuszcza stado goryli',
 ' mistrzowsko przeprowadzona narracja sprawia, że nikt z widzów nie podejrzewa o morderstwo goryla palmera, williamsa',
 ' moi goryle się pochowali',
 'gdy dwa goryle płci męskiej spotkają się na ścieżce, stają wyprostowane, zaczynają walić się w piersi i ryczeć']

In [9]:
useRand = False
alreadyUsed = []

matchExact = 1
matchBase = 1
penalty = -100

def score(quote, query):
    numWords = 0
    if quote in alreadyUsed:
        return penalty
    if query in quote:
        return penalty
    for w in quote.split():
        if w in query:
            numWords+=1
    if numWords >= 0.75 * len(quote.split()) and len(quote.split()) < 2* len(query.split()):
        return penalty
    me = matchExact * len([w for w in query.split() if w in quote])
    basedQuote = [lemmasDict.get(w) for w in quote.split()]
    mb = matchBase * len([w for w in query.split() if lemmasDict.get(w) in basedQuote])
    return me + mb

def ranking(quotes, query):
    return [(q, score(q, query)) for q in quotes]

def choose(ranked):
    if len(ranked) == 0:
        ranked = list(zip(defaultAnswers, len(defaultAnswers)*[1]))
    if useRand:
        quotes, scores = zip(*ranked)
        ans = random.choices(quotes, weights=scores)[0]
    else:
        ans = max(ranked, key=lambda x:x[1])[0]
    alreadyUsed.append(ans)
    return ans

In [10]:
a, b  =zip(*ranking(searchQuotes("goryle"), "płci"))
b

(0, 0, 0, 0, -100)

In [ ]:
while True:
    query = input(": ")
    if query == "end":
        break
    query = query.replace('?', '').replace('!', '').replace('.', '')
    #print(query)
    foudQuotes = searchQuotes(query)
    ranked = ranking(foudQuotes, query)
    ans = choose(ranked)
    print(ans)


: Jak napisać konspekt z etyki?
 aby dowiedzieć się, na czym polega rzetelna praca w tym zawodzie, wystarczy zapoznać się z kodeksem etyki dziennikarskiej napisanym przez bł
